## 0. Libarary 불러오기 및 경로설정

In [41]:
import os
import random
from collections import defaultdict
from enum import Enum
from typing import Tuple, List
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from tqdm import tqdm
import torch.optim as optim
from PIL import Image
import os
from importlib import import_module
from torch.utils.data import Dataset, Subset, random_split
import multiprocessing
from torchvision.transforms import (
    Resize,
    ToTensor,
    Normalize,
    Compose,
    CenterCrop,
    ColorJitter,
)
from dataset import BaseAugmentation

In [34]:
data_dir = '/data/ephemeral/home/data/train/images'

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [35]:
from dataset import JayDataset1, MaskModelDataset

dataset = MaskModelDataset(data_dir)
class MyArg:
    resize = [128, 96]

transform = BaseAugmentation(MyArg(), dataset)
dataset.set_transform(transform)

mask_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=20,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=True,
        pin_memory=use_cuda,
        drop_last=False,
    )


In [37]:
from models.mask_model import Resnet34

class JayCustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.model_wear = Resnet34(num_classes=2)
        self.model_correct = Resnet34(num_classes=2)
        self.wear_path = '/data/ephemeral/home/project/repo/level1-imageclassification-cv-06/best_model/mask_model_wear_notwear.pth'
        self.correct_path = '/data/ephemeral/home/project/repo/level1-imageclassification-cv-06/best_model/mask_model_correct_incorrect.pth'

        # 모델 가중치를 로드한다.
        self.model_wear.load_state_dict(torch.load(self.wear_path, map_location=device))
        self.model_correct.load_state_dict(torch.load(self.correct_path, map_location=device))


    def forward(self, x):
        x1 = self.model_wear(x)
        x1 = torch.argmax(x1, -1)

        for i in range(len(x1)):
            if x1[i] == 1: #쓴 경우
                x2 = self.model_correct(x[i].unsqueeze(0))
                x2 = torch.argmax(x2, -1)
                if x2[0] == 0: #마스크가 정상인 경우
                    x1[i] = 0
                else:
                    x1[i] = 1
            else: #안쓴 경우
                x1[i] = 2
        return x1

In [43]:
model = JayCustomModel().cuda()

model.eval()

labels = []
preds = []
for epoch in tqdm(range(2)):
    for index, batch in enumerate(mask_loader):
        image, label = batch
        image = image.cuda()
        label = label.long().cuda()
        labels.extend(label.cpu().numpy())
        pred = model(image)
        preds.extend(pred.cpu().numpy())
        if index > 5:
            break



/data/ephemeral/home/jae-env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/jae-env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


In [44]:
recall = recall_score(labels, preds, average='macro')
f1 = f1_score(labels, preds, average='macro')
conf_matrix = confusion_matrix(labels, preds)

print("Recall:", recall)
print("f1:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Recall: 0.9771510641075859
f1: 0.9663356787444378
Confusion Matrix:
[[204   3   0]
 [  0  35   2]
 [  0   0  36]]
